<a href="https://colab.research.google.com/github/Luzinsan/OPD_neuro_practice2/blob/main/Handwritten_Chinese_classes32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import time
import PIL.Image as Image
import matplotlib.pylab as plt
import tensorflow as tf
import os
import zipfile
import shutil

!git clone https://github.com/AI-FREE-Team/Traditional-Chinese-Handwriting-Dataset.git
OutputFolder = '/content/Handwritten_Data'
!rm -rf '/content/Handwritten_Data'

fatal: destination path 'Traditional-Chinese-Handwriting-Dataset' already exists and is not an empty directory.


In [2]:
SIZE = 32 # рассматриваем 32 класса

In [3]:
CompressedFiles = []

os.chdir('/content/Traditional-Chinese-Handwriting-Dataset/data')

for item in os.listdir():  
  if item.endswith('.zip'): # Check for ".zip" extension.
    file_path = os.path.abspath(item) # Get full path of the compressed file. 
    CompressedFiles.append(file_path)

for file in CompressedFiles:     
  # Construct a ZipFile object with the filename, and then extract it.
  zip_ref = zipfile.ZipFile(file).extractall(OutputFolder) 
  
  source_path = OutputFolder + '/cleaned_data(50_50)'
  img_list = os.listdir(source_path)

  for img in img_list:
      shutil.move(source_path + '/' + img, OutputFolder) # Move a file to another location. 
  
  shutil.rmtree(OutputFolder + '/cleaned_data(50_50)') 
  #print(f'Decompress successfully {file} ......')
  #print( 'Moving images according to traditional Chinese characters......' )

ImageList = os.listdir(OutputFolder)
ImageList = [img for img in ImageList if len(img)>1]
WordList = list(set([w.split('_')[0] for w in ImageList]))[:SIZE]

for w in WordList:
  try:
    os.chdir(OutputFolder) # Change the current working directory to OutputPath.
    os.mkdir(w) # Create the new word folder in OutputPath.
    MoveList = [img for img in ImageList if w in img]
                
  except: 
    os.chdir(OutputFolder)
    MoveList = [img for img in ImageList if w in img ]
  
  finally:            
    for img in MoveList:
      old_path = OutputFolder + '/' + img
      new_path = OutputFolder + '/' + w + '/' + img
      shutil.move( old_path, new_path )

#print( 'Data Deployment completed.' )

In [4]:
a=0
b=0

for item in os.listdir(OutputFolder):
  if (os.path.isdir(item)):  
    a += 1
    for i in os.listdir(OutputFolder + '/' + item):
      b +=1

#print ('Всего: ' + str(a) + ' слов (папка) / Всего: ' + str(b) + ' образцов')
#print ('В среднем каждое слово содержит: ' + str (b / a) + ' образцов')

In [5]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.1)
train_dataset = image_generator.flow_from_directory(str(OutputFolder), class_mode='sparse', batch_size=5, target_size=(50, 50), subset='training')
valid_dataset = image_generator.flow_from_directory(str(OutputFolder), class_mode='sparse', batch_size=5, target_size=(50, 50), subset='validation')

Found 1527 images belonging to 32 classes.
Found 157 images belonging to 32 classes.


In [6]:
for image_b, label_b in train_dataset:
  #print("Image batch shape: ", image_b.shape)
  #print("Label batch shape: ", label_b.shape)
  image_batch = image_b
  label_batch = label_b
  #print(label_batch)
  break

In [7]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=16,  kernel_size=3, activation='relu', padding= 'same' , input_shape=(50,50,3)),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
  tf.keras.layers.Conv2D(filters=32,  kernel_size=3, activation='relu', padding= 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
  tf.keras.layers.Conv2D(filters=64,  kernel_size=3, activation='relu', padding= 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),
  tf.keras.layers.Conv2D(filters=128,  kernel_size=3, activation='relu', padding= 'same'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(SIZE, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 50, 50, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 25, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 64)         0

In [8]:
model.compile(
  optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.9),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)
EPOCHS = 10
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=EPOCHS)

Epoch 1/10
306/306 [==============================] - 10s 30ms/step - loss: 4.7684 - accuracy: 0.0321 - val_loss: 3.5643 - val_accuracy: 0.0446
Epoch 2/10
306/306 [==============================] - 8s 26ms/step - loss: 3.4282 - accuracy: 0.0616 - val_loss: 3.2468 - val_accuracy: 0.1210
Epoch 3/10
306/306 [==============================] - 8s 26ms/step - loss: 3.1499 - accuracy: 0.1074 - val_loss: 2.8338 - val_accuracy: 0.1338
Epoch 4/10
306/306 [==============================] - 8s 26ms/step - loss: 2.6207 - accuracy: 0.2364 - val_loss: 2.0614 - val_accuracy: 0.3758
Epoch 5/10
306/306 [==============================] - 8s 27ms/step - loss: 1.9208 - accuracy: 0.4139 - val_loss: 1.2902 - val_accuracy: 0.6178
Epoch 6/10
306/306 [==============================] - 8s 26ms/step - loss: 1.2295 - accuracy: 0.6189 - val_loss: 0.9253 - val_accuracy: 0.6815
Epoch 7/10
306/306 [==============================] - 8s 26ms/step - loss: 0.8333 - accuracy: 0.7413 - val_loss: 0.6753 - val_accuracy: 0.770

In [10]:
model.compile(
  optimizer=tf.keras.optimizers.Adadelta(learning_rate = 0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy']
)
EPOCHS = 50
history = model.fit(train_dataset,
                    validation_data=valid_dataset,
                    epochs=EPOCHS)

Epoch 1/50
306/306 [==============================] - 9s 28ms/step - loss: 0.2251 - accuracy: 0.9234 - val_loss: 0.3691 - val_accuracy: 0.8981
Epoch 2/50
306/306 [==============================] - 8s 27ms/step - loss: 0.2354 - accuracy: 0.9253 - val_loss: 0.3623 - val_accuracy: 0.8981
Epoch 3/50
306/306 [==============================] - 8s 27ms/step - loss: 0.2035 - accuracy: 0.9273 - val_loss: 0.3563 - val_accuracy: 0.8981
Epoch 4/50
306/306 [==============================] - 8s 28ms/step - loss: 0.2075 - accuracy: 0.9345 - val_loss: 0.3510 - val_accuracy: 0.8981
Epoch 5/50
306/306 [==============================] - 8s 27ms/step - loss: 0.2042 - accuracy: 0.9319 - val_loss: 0.3458 - val_accuracy: 0.8981
Epoch 6/50
306/306 [==============================] - 8s 27ms/step - loss: 0.2107 - accuracy: 0.9267 - val_loss: 0.3405 - val_accuracy: 0.8981
Epoch 7/50
306/306 [==============================] - 8s 28ms/step - loss: 0.1967 - accuracy: 0.9345 - val_loss: 0.3360 - val_accuracy: 0.8981

In [11]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([min(plt.ylim()),max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()